# How to get MOC data from VESPA in Python using ElasticSearch


## Import required modules

In [1]:
from elasticsearch import Elasticsearch

## Connect to the ElasticSearch server

In [2]:
es = Elasticsearch('http://voparis-elasticsearch.obspm.fr:9200')

## Get a list of the indices

In [3]:
print("Indices list:\n")
for index in es.indices.get(index='*'):
    print(index)

Indices list:

.ds-.monitoring-es-8-mb-2022.10.20-000001
.ds-.monitoring-es-8-mb-2022.10.23-000002
.ds-.monitoring-es-8-mb-2022.10.26-000003
.ds-.monitoring-es-8-mb-2022.10.29-000004
.ds-.monitoring-es-8-mb-2022.11.01-000005
.ds-.monitoring-es-8-mb-2022.11.04-000006
.ds-.monitoring-es-8-mb-2022.11.07-000007
.ds-.monitoring-es-8-mb-2022.11.10-000008
.ds-.monitoring-es-8-mb-2022.11.13-000009
.ds-.monitoring-es-8-mb-2022.11.16-000010
.ds-.monitoring-es-8-mb-2022.11.19-000011
.ds-metricbeat-8.4.3-2022.10.20-000001
.ds-metricbeat-8.4.3-2022.11.19-000002
exoplanet
exoplanet_index
moc-index
obsfacility_index
test_chloe
vespa-index
vespa_index


## Get the number of items in moc-index in Vvex, grouped by target name

In [62]:
indexName = 'moc-index'

In [67]:
query = {
    "bool": {
         "must": [
            {
              "bool": {
                "should": [
                  {
                    "term": {
                      "service_title.keyword": {
                        "value": "vvex"
                      }
                    }
                  }
                ],
                "minimum_should_match": 1
              }
            }
      ],
      "filter": [],
      "should": [],
      "must_not": []
    }
}

aggs = {
    "agg-test-terms-count" : {
        "terms": {
            "field" : "target_name.keyword",
            "size": 5
        },
    }
}


page = es.search(
    index=indexName,
    query=query,
    size=0,
    fields=["*"],
    aggs = aggs,
)

for bucket in page["aggregations"]["agg-test-terms-count"]["buckets"] :
    print(bucket["key"],":",bucket["doc_count"])

Number of hits:  {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': None, 'hits': []}
Venus : 45304
Earth : 292
Star : 16
Mars : 8
Mercury : 8


## Check if an index has a coverage field

In [54]:
def hasCoverage(es, indexName) :
    return "coverage" in es.indices.get_mapping(index=indexName)[indexName]["mappings"]["properties"]

In [59]:
if not hasCoverage(es,indexName) :
    print("There is no coverage in the index ", indexName)

## Import modules for Aladin and MOCs

In [60]:
from mocpy import MOC
from ipyaladin import Aladin

## How to load a MOC in Aladin

In this section we will see how to load a MOC in Aladin.

### Get the data

In [93]:
query = {
    "bool": {
         "must": [
            {
              "bool": {
                "must" : [
                    {
                        "exists" : {
                            "field" : "coverage"
                        }
                    }
                ],
                "should": [
                  {
                    "term": {
                      "service_title.keyword": {
                        "value": "vvex"
                      }
                    }
                  }
                ],
                "minimum_should_match": 1
              }
            }
      ],
      "filter": [],
      "should": [],
      "must_not": []
    }
}


page = es.search(
    index=indexName,
    query=query,
    size=10,
    fields=["*"],
)

data = []
for document in page["hits"]["hits"] :
    data.append(document["_source"])
    print(document["_source"]["granule_uid"])

VV0086_19G
VV0117_02G
VV0085_15C
VV0084_11C
VV0084_14C
VV0084_10G
VV0084_03G
VV0083_03C
VV0082_13C
VV0082_13G


### Load Aladin and add the data that was fetched just above

Here, and in the following examples, we are using the HiPS (Hierarchical Progressive Surveys) of Venus.
Other HiPS can be found here http://voparis-srv-paris.obspm.fr/vo/planeto/hips/.

In [89]:
aladin = Aladin(
    coo_frame="body",
    survey="http://voparis-srv-paris.obspm.fr/vo/planeto/hips/CDS_P_Venus_Magellan_C3-MDIR-2025m/"
)
aladin

Aladin(coo_frame='body', options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_sur…

### Load the MOCs

First, for each MOC, we create a MOC object with mocpy and then we convert it to a dictionnary, since it's the format that ipyaladin expects in the function add_moc_from_dict.

In [94]:
for item in data :
    mocObject = MOC.from_str(item["coverage"])
    jsonMoc = mocObject.serialize(format='json', optional_kw_dict=None, pre_v2=False)
    aladin.add_moc_from_dict(jsonMoc, {'opacity' : 0.5, 'name' : item["granule_uid"]})

We center the view on one of the MOC

In [95]:
mocCenter = MOC.from_str(data[0]["coverage"]).barycenter()
aladin.target = mocCenter.to_string()

## Union and intersection of MOCs

In this section, we are going to see how to search specific MOCs.
Here we only consider MOCs in the Northern Hemisphere, whose dataproduct_type is the spectral cube and whose observation minimim local time was before 20 P.M.

Then, we will group those MOCs in three categories, corresponding to the channel_id field associated to them.
And finally, we will search all the elements whose MOC is in a MOC defined as the intersection of those three groups.

### Get the data

In [114]:
query = {
    "bool": {
         "must": [
             {
                "range" : {
                    "c1min" : {
                        "gte" : 0,
                        "lte" : 360
                    }
                }
             },
             {
                "range" : {
                    "c1max" : {
                        "gte" : 0,
                        "lte" : 360
                    }
                }
             },
             {
                "range" : {
                    "c2min" : {
                        "gte" : 0,
                        "lte" : 90
                    }
                }
             },
             {
                "range" : {
                    "c2max" : {
                        "gte" : 0,
                        "lte" : 90
                    }
                }
             },
             {
                "range" : {
                    "local_time_min" : {
                        "lte" : 20
                    }
                }
             },
             {
                "term" : {
                    "dataproduct_type" : "sc"
                }
            },
            {
                "term": {
                    "service_title.keyword": {
                    "value": "vvex"
                  }
                 }
             },
             {
                "exists" : {
                    "field" : "coverage"
                }
            }
        ]
    }
}


page = es.search(
    index=indexName,
    query=query,
    size=50,
    fields=["*"],
)

data = []
for document in page["hits"]["hits"] :
    data.append(document["_source"])
    print(document["_source"]["granule_uid"])

print("\n",len(data)," results found")

VI0041_02C
VT0033_00C
VI0060_05C
VV0041_02C
VI0058_00C
VV0058_00C
VT0041_03C
VI0047_00C
VV0060_05C
VV0047_00C
VV0062_00C
VI0062_00C
VT0058_00C
VT0060_01C
VT0062_00C
VT0047_00C

 16  results found


### Group the MOCs by channel_id

Here we are using a dictionnary: each key is a channel_id value, the value associated to it is an array containing all the MOCs whose channel_id is equal to this key.
In the following block of code we are filling this dictionnary.

In [116]:
mocGroups = {"VIRTIS_M_VIS":[], "VIRTIS_M_IR":[], "VIRTIS_H":[]}
for element in data :
    mocGroups[element["channel_id"]].append(MOC.from_str(element["coverage"]))

For each channel_id we are now using mocpy to make an union of all the MOCs in the array corresponding to this channel_id.
So we add the first element to mocUnion, we remove it from the array, then for each element of the array we make an union between mocUnion and this element, and finally, we replace the array of MOCs with the MOC mocUnion at the corresponding key.

So, the following block of code converts a dictionnary of MOCs array to a dictionnary of MOCs that covers the same area as the array of MOCs that was previously at the same key.

In [118]:
keys = list(mocGroups.keys())
for key in keys :
    if(len(mocGroups[key]) > 0):
        mocUnion = mocGroups[key][0]
        mocGroups[key].pop(0)
        for moc in mocGroups[key] :
            mocUnion = mocUnion.union(moc)
        mocGroups[key] = mocUnion
    else :
        mocGroups.pop(key)

In [119]:
aladin2 = Aladin(
    coo_frame='body',
    survey="http://voparis-srv-paris.obspm.fr/vo/planeto/hips/CDS_P_Venus_Magellan_C3-MDIR-2025m/"
)
aladin2

Aladin(coo_frame='body', options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_sur…

In [120]:
for group, moc in mocGroups.items() :
    jsonMoc = moc.serialize(format='json', optional_kw_dict=None, pre_v2=False)
    aladin2.add_moc_from_dict(jsonMoc, {'opacity' : 0.5, 'name' : group})

In [121]:
mocCenter = list(mocGroups.values())[0].barycenter()
aladin2.target = mocCenter.to_string()

### Intersection of MOCs

Here, we get the intersection of the three MOCs in mocGroups

In [122]:
mocList = []
for moc in mocGroups.values() :
    mocList.append(moc)
    
if(len(mocGroups) > 0):
    mocIntersection = mocList[0]
    mocList.pop(0)
    for moc in mocList :
        mocIntersection = mocIntersection.intersection(moc)

We can plot this intersection in Aladin if needed:

In [123]:
jsonMoc = mocIntersection.serialize(format='json', optional_kw_dict=None, pre_v2=False)
aladin2.add_moc_from_dict(jsonMoc, {'opacity' : 0.5, 'name' : 'intersection'})

In [124]:
mocCenter = mocIntersection.barycenter()
aladin2.target = mocCenter.to_string()

Then, for each item in the data that we got in the "Get the data" section, we can get the intersection between the MOC associated to this item and the MOC corresponding to the intersection of the three groups of MOCs (mocIntersection).

If this intersection is not empty, then this item is in mocIntersection.
So we get a list of items whose MOC intersects with the MOC of other items with the two other types of channel_id

In [125]:
results = []
for item in data :
    moc = MOC.from_str(item["coverage"])
    if not moc.intersection(mocIntersection).empty() :
        results.append(item)        

In [126]:
print(len(results), " results found:")
for item in results :
    print(item["granule_uid"])

15  results found:
VI0041_02C
VI0060_05C
VV0041_02C
VI0058_00C
VV0058_00C
VT0041_03C
VI0047_00C
VV0060_05C
VV0047_00C
VV0062_00C
VI0062_00C
VT0058_00C
VT0060_01C
VT0062_00C
VT0047_00C
